In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [ ]:
# Для воспроизводимости

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# RNN intro

Давайте разберемся что из себя вообще представляют рекуррентные нейронные сети в самом простом виде.

<img src="./pics/rnn.png" width="90%">

В самом простом виде для одного входного вектора $x_{(t)}$ и одного слоя рекуррентной сети справедливо такое соотношение:

$$y_{(t)} = \phi (x_{(t)}^T \cdot w_x + y_{(t-1)}^T \cdot w_y + b)$$

где 
* $x(t)$ -- входной вектор на текущем шаге
* $y(t)$ -- выходной вектор на текущем шаге
* $w_x$ -- вектор весов нейронов для входа
* $w_y$ -- вектор весов нейронов для выхода
* $y(t-1)$ -- выходной вектор с прошлого шага. Для шага 0 этот вектор нулевой
* $b$ -- байес (bias)
* $\phi$ -- обозначение для функции активации, например ReLU


Есть понятие **hidden_state** ( $h(t)$ ) -- это "память" рекуррентной ячейки.

В общем случае $h_{(t)} = f(h_{(t-1)}, x_{(t)})$, но выход также $y{(t)} = f(h{(t-1)}, x{(t)})$.

В данном случае $h(t) == y(t)$, но на практике используются более сложные архитектуры и в них **hidden_state** не совпадает с непосредственным выходом нейросетки.

------

## Напишем свою простую RNN сеть

Снова немножко математики чтобы привести формулу выше к более удобному виду.

Представим, что на вход подается не один вектор $x_{(t)}$, а целый мини-батч размера $m$ таких векторов $X_{(t)}$, соответственно все дальнейшие размышления мы уже производим в матричном виде:

$$ Y_{(t)} = \phi(X_{(t)}^T \cdot W_x + Y_{(t-1)}^T \cdot W_y + b) = \phi([X_{(t)} Y_{(t-1)}] \cdot W + b) $$
где
$$ W = [W_x W_y]^T $$

*Операция в скобках квадратных -- конкатенация матриц

По размерностям:
* $Y_{(t)}$ -- матрица [$m$ x n_neurons]
* $X_{(t)}$ -- матрица [$m$ x n_features]
* $b$ -- вектор длины n_neurons
* $W_x$ -- веса между входами и нейронами размерностью [n_features x n_neurons]
* $W_y$ -- веса связей с прошлым выходом размерностью [n_neurons x n_neurons]

In [ ]:
reset_graph()

# Напишем нейронку прямо как на картинке в самом верху с 5-ю нейронами
# На вход будем подавать векторы длины 3
n_features = 3
n_neurons = 5

# С текушей имплементацией наша нейронка делает всего 2 шага
X0 = tf.placeholder(tf.float32, [None, n_features])
X1 = tf.placeholder(tf.float32, [None, n_features])

Wx = tf.Variable(tf.random_normal(shape=[n_features, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

# Здесь в качестве функции phi берем гиперболический тангенс
Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

init = tf.global_variables_initializer()

In [ ]:
# Будем подавать на вход мини батчи размером 4
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]])  # t = 0
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]])  # t = 1

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [ ]:
Y0_val

In [ ]:
Y1_val

## Задание

Напишите то же самое, но использовав всего одно матричное перемножение на каждом шаге (см формулу в объяснении выше)

In [ ]:
reset_graph()

X0 = tf.placeholder(tf.float32, [None, n_features])
X1 = tf.placeholder(tf.float32, [None, n_features])

Wx = tf.Variable(tf.random_normal(shape=[n_features, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
#W = tf.Variable(tf.random_normal(shape = [n_features + n_neurins, n_neurons + n_neurones], dtype = tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype = tf.float32))

Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(tf.concat([X1, Y0], 1), tf.concat([Wx, Wy], 0)) + b)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    Y0_val_1, Y1_val_1 = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [ ]:
Y0_val

# Dynamic_rnn

В tf есть функция `tf.contrib.rnn.static_rnn` которая создает для каждого unrolling'а (т.е. каждого батча на входе) отдельную ячейку того типа, который мы ей передадим. В данном случае наша имплементация совпадает с имплементацией `tf.contrib.rnn.BasicRNNCell` в tf. Это все не очень круто, если требуется сделать большое число шагов -- у нас попросту может закончится память, если мы вдруг решим делать backprop. Поэтому к счастью есть другой путь -- это `dynamic_rnn`.

Сделаем весь предыдущий пример с помощью `dynamic_rnn`.

In [ ]:
n_steps = 2
n_features = 3
n_neurons = 5

reset_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_features])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

In [ ]:
seq_length = tf.placeholder(tf.int32, [None])
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32,
                                    sequence_length=seq_length)

In [ ]:
init = tf.global_variables_initializer()
X_batch = np.array([
        # step 0     step 1
        [[0, 1, 2], [9, 8, 7]], # instance 1
        [[3, 4, 5], [0, 0, 0]], # instance 2 (padded with zero vectors)
        [[6, 7, 8], [6, 5, 4]], # instance 3
        [[9, 0, 1], [3, 2, 1]], # instance 4
    ])

# параметр с истинной длиной последовательностей
seq_length_batch = np.array([2, 1, 2, 2])

In [ ]:
with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run(
        [outputs, states], feed_dict={X: X_batch, seq_length: seq_length_batch})

In [ ]:
print(outputs_val.shape)
print(states_val.shape)

In [ ]:
# заметим, что у второго инпута есть нули в final state
print(outputs_val)

In [ ]:
# а тут уже нет нулей;
# так происходит благодаря наличию параметра seq_length
print(states_val)

# Генерация имен

А теперь попробуем понять что можно со всем этим вышеперечисленным делать полезного.

_Teaser:_

* Сложно придумать имя для переменной? Но куда сложнее придумать хорошее имя для человека.
  Поэтому давайте напишем нейронку, которая сделает это за нас.
* Набор данных содержит ~ 8 тыс человеческих имен из разных культур [в латинской расшифровке]
* Цель (игрушечная проблема): изучить генеративную модель по именам.

In [ ]:
start_token = " "

with open("names") as f:
    names = f.read()[:-1].split('\n')
    names = [start_token + name.lower() for name in names]

In [ ]:
print('n samples = ', len(names))
for x in names[::1000]:
    print(x.strip().capitalize())

# Обработка текста

Рассмотрим все буквы без учета регистра + ')' -- конец имени

In [ ]:
# Найдем все уникальные символы
# Без учета регистра

token_set = set()
for name in names:
    for letter in name:
        token_set.add(letter)


token_set.add(')')
tokens = list(token_set)
tokens.sort()

print('n_tokens = ', len(tokens))

In [ ]:
names[0]

In [ ]:
tokens

In [ ]:
#!token_to_id = < словарь символов -> их id (index в tokens list)>
token_to_id = {t: i for i, t in enumerate(tokens)}

#!id_to_token = < словарь айдишников -> соответствующие символы >
id_to_token = {i: t for i, t in enumerate(tokens)}

## Построим распределение длин всех имен

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(list(map(len, names)))

# Посмотрим какая максимальная длина у имени в этом датасете
MAX_LEN = min([60, max(list(map(len, names)))])-1

print(MAX_LEN)

### Переведем все символы в их id

In [ ]:
list(map(token_to_id.get, 'anna' + ')'))

In [ ]:
names_ix = list(map(lambda name: list(map(token_to_id.get, name + ')')), names))


# Добьем нулями короткие имена до MAX_LEN и усечем слишком длинные
for i in range(len(names_ix)):
    names_ix[i] = names_ix[i][:MAX_LEN+1] #crop too long
    
    if len(names_ix[i]) < MAX_LEN+1:
        names_ix[i] += [token_to_id[" "]]*(MAX_LEN+1 - len(names_ix[i])) #pad too short
        
assert len(set(map(len, names_ix))) == 1

names_ix = np.array(names_ix)

In [ ]:
names_ix[:10]

## Генерилка батчей

In [ ]:
def sample_batch(data, batch_size):
    
    rows = data[np.random.randint(0, len(data), size=batch_size)]
    x = rows[:, :-1]
    y = rows[:, 1:]
    
    count = lambda r: np.sum([id_to_token[t] != ' ' for t in r])
    lengths = list(map(count, x))
    
    return x, y, lengths

In [ ]:
x, y, length = sample_batch(names_ix, 10)
y.shape

In [ ]:
x

In [ ]:
y

In [ ]:
length

# Входы сетки

In [ ]:
reset_graph()

# подразумевается, что размерность X [batch_size, max_length];
X = tf.placeholder(tf.int32, [None, None], name= 'X')
y = tf.placeholder(tf.int32, [None, None], name = 'y')
lengths = tf.placeholder(tf.int32, [None], name = 'lengths')
learning_rate_ph = tf.placeholder(dtype=tf.float32, shape=[])

In [ ]:
n_neurons = 60
embedding_size = 8
vocabulary_size = len(tokens)

n_steps = MAX_LEN # Этот параметр совпадает
                  # с максимальной длиной последовательности, 
                  # которая может быть подана на вход
                  # иначе говоря, это unrollings

# для входной последовательности создаем матрицу эмбеддингов
embedding_mtx = tf.get_variable(name='embeddings', shape=[vocabulary_size, embedding_size])

# достаем из матрицы эмбеддингов нужные нам векторы X
embed = tf.nn.embedding_lookup(embedding_mtx, X)


cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.tanh)
rnn_outputs, states = tf.nn.dynamic_rnn(cell=cell, inputs=embed,
                                        sequence_length=lengths, dtype=tf.float32)

# получаем ненормированное распределение по классам 
# для каждого анроллинга в каждом сэмле в батче
pred_logits = tf.layers.dense(inputs=rnn_outputs, units=vocabulary_size, name='output_projection')

# кодируем one-hot классы, т.к. это тоже нужно функции лосса
labels_one_hot = tf.one_hot(y, depth=vocabulary_size, dtype=tf.float32)

# считаем илололосс
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=labels_one_hot,
    logits=pred_logits)
    
loss = tf.reduce_mean(stepwise_cross_entropy)

pred_probas = tf.nn.softmax(pred_logits)

# берем максимум по оси, соответствующей количеству классов
# получаем матрицу размера [batch_size, num_steps]
prediction = tf.argmax(pred_probas, axis=2)

train_op = tf.train.AdamOptimizer(learning_rate_ph).minimize(loss)

# берем распределение вероятностей только для последнего символа в каждом сэмпле
# это потребуется для генерации
last_word_probas = pred_probas[:, -1]

In [ ]:
reset_graph()

# подразумевается, что размерность X [batch_size, max_length];
X = tf.placeholder(tf.int32, [None, None], name= 'X')
y = tf.placeholder(tf.int32, [None, None], name = 'y')
lengths = tf.placeholder(tf.int32, [None], name = 'lengths')
learning_rate_ph = tf.placeholder(dtype=tf.float32, shape=[])

n_neurons = 60
n_layers = 2
embedding_size = 8
vocabulary_size = len(tokens)

n_steps = MAX_LEN # Этот параметр совпадает
                  # с максимальной длиной последовательности, 
                  # которая может быть подана на вход
                  # иначе говоря, это unrollings

# для входной последовательности создаем матрицу эмбеддингов
embedding_mtx = tf.get_variable(name='embeddings', shape=[vocabulary_size, embedding_size])

# достаем из матрицы эмбеддингов нужные нам векторы X
embed = tf.nn.embedding_lookup(embedding_mtx, X)


#cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.tanh)
#rnn_outputs, states = tf.nn.dynamic_rnn(cell=cell, inputs=embed,
#                                        sequence_length=lengths, dtype=tf.float32)

# Как ненапряжно замутить глубокую рекуррентную нейросеть


layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                      activation=tf.nn.relu) for layer in range(n_layers)]
cell = tf.contrib.rnn.MultiRNNCell(layers)
#outputs, states = tf.nn.dynamic_rnn(cell = multi_layer_cell, inputs = X, dtype=tf.int32)
rnn_outputs, states = tf.nn.dynamic_rnn(cell, tf.nn.embedding_lookup(embedding_mtx, X), dtype = tf.float32, scope='rnnlm')
# получаем ненормированное распределение по классам 
# для каждого анроллинга в каждом сэмле в батче
pred_logits = tf.layers.dense(inputs=rnn_outputs, units=vocabulary_size, name='output_projection')

# кодируем one-hot классы, т.к. это тоже нужно функции лосса
labels_one_hot = tf.one_hot(y, depth=vocabulary_size, dtype=tf.float32)

# считаем илололосс
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=labels_one_hot,
    logits=pred_logits)
    
loss = tf.reduce_mean(stepwise_cross_entropy)

pred_probas = tf.nn.softmax(pred_logits)

# берем максимум по оси, соответствующей количеству классов
# получаем матрицу размера [batch_size, num_steps]
prediction = tf.argmax(pred_probas, axis=2)

train_op = tf.train.AdamOptimizer(learning_rate_ph).minimize(loss)

# берем распределение вероятностей только для последнего символа в каждом сэмпле
# это потребуется для генерации
last_word_probas = pred_probas[:, -1]

In [ ]:
reset_graph()

# подразумевается, что размерность X [batch_size, max_length];
X = tf.placeholder(tf.int32, [None, None], name= 'X')
y = tf.placeholder(tf.int32, [None, None], name = 'y')
lengths = tf.placeholder(tf.int32, [None], name = 'lengths')
learning_rate_ph = tf.placeholder(dtype=tf.float32, shape=[])


n_neurons = 240
embedding_size = 18
vocabulary_size = len(tokens)

n_steps = MAX_LEN # Этот параметр совпадает
                  # с максимальной длиной последовательности, 
                  # которая может быть подана на вход
                  # иначе говоря, это unrollings

# для входной последовательности создаем матрицу эмбеддингов
embedding_mtx = tf.get_variable(name='embeddings', shape=[vocabulary_size, embedding_size])

# достаем из матрицы эмбеддингов нужные нам векторы X
embed = tf.nn.embedding_lookup(embedding_mtx, X)


cell = tf.contrib.rnn.LSTMCell(num_units=n_neurons, activation=tf.nn.tanh)
rnn_outputs, states = tf.nn.dynamic_rnn(cell=cell, inputs=embed,
                                        sequence_length=lengths, dtype=tf.float32)

# получаем ненормированное распределение по классам 
# для каждого анроллинга в каждом сэмле в батче
pred_logits = tf.layers.dense(inputs=rnn_outputs, units=vocabulary_size, name='output_projection')

# кодируем one-hot классы, т.к. это тоже нужно функции лосса
labels_one_hot = tf.one_hot(y, depth=vocabulary_size, dtype=tf.float32)

# считаем илололосс
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=labels_one_hot,
    logits=pred_logits)
    
loss = tf.reduce_mean(stepwise_cross_entropy)

pred_probas = tf.nn.softmax(pred_logits)

# берем максимум по оси, соответствующей количеству классов
# получаем матрицу размера [batch_size, num_steps]
prediction = tf.argmax(pred_probas, axis=2)

train_op = tf.train.AdamOptimizer(learning_rate_ph).minimize(loss)

# берем распределение вероятностей только для последнего символа в каждом сэмпле
# это потребуется для генерации
last_word_probas = pred_probas[:, -1]

In [ ]:
reset_graph()

# подразумевается, что размерность X [batch_size, max_length];
X = tf.placeholder(tf.int32, [None, None], name= 'X')
y = tf.placeholder(tf.int32, [None, None], name = 'y')
lengths = tf.placeholder(tf.int32, [None], name = 'lengths')
learning_rate_ph = tf.placeholder(dtype=tf.float32, shape=[])

n_neurons = 60
n_layers = 2
embedding_size = 8
vocabulary_size = len(tokens)

n_steps = MAX_LEN # Этот параметр совпадает
                  # с максимальной длиной последовательности, 
                  # которая может быть подана на вход
                  # иначе говоря, это unrollings

# для входной последовательности создаем матрицу эмбеддингов
embedding_mtx = tf.get_variable(name='embeddings', shape=[vocabulary_size, embedding_size])

# достаем из матрицы эмбеддингов нужные нам векторы X
embed = tf.nn.embedding_lookup(embedding_mtx, X)


#cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.tanh)
#rnn_outputs, states = tf.nn.dynamic_rnn(cell=cell, inputs=embed,
#                                        sequence_length=lengths, dtype=tf.float32)

# Как ненапряжно замутить глубокую рекуррентную нейросеть


layers = [tf.contrib.rnn.LSTMCell(num_units=n_neurons,
                                      activation=tf.nn.relu) for layer in range(n_layers)]
cell = tf.contrib.rnn.MultiRNNCell(layers)
#outputs, states = tf.nn.dynamic_rnn(cell = multi_layer_cell, inputs = X, dtype=tf.int32)
rnn_outputs, states = tf.nn.dynamic_rnn(cell, tf.nn.embedding_lookup(embedding_mtx, X), dtype = tf.float32, scope='lstmlm')
# получаем ненормированное распределение по классам 
# для каждого анроллинга в каждом сэмле в батче
pred_logits = tf.layers.dense(inputs=rnn_outputs, units=vocabulary_size, name='output_projection')

# кодируем one-hot классы, т.к. это тоже нужно функции лосса
labels_one_hot = tf.one_hot(y, depth=vocabulary_size, dtype=tf.float32)

# считаем илололосс
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=labels_one_hot,
    logits=pred_logits)
    
loss = tf.reduce_mean(stepwise_cross_entropy)

pred_probas = tf.nn.softmax(pred_logits)

# берем максимум по оси, соответствующей количеству классов
# получаем матрицу размера [batch_size, num_steps]
prediction = tf.argmax(pred_probas, axis=2)

train_op = tf.train.AdamOptimizer(learning_rate_ph).minimize(loss)

# берем распределение вероятностей только для последнего символа в каждом сэмпле
# это потребуется для генерации
last_word_probas = pred_probas[:, -1]

In [ ]:
Class tf.contrib.rnn.LSTMCell
Class tf.nn.rnn_cell.LSTMCell

### Как можно узнать о параметрах, которые тренируются

In [ ]:
tf.trainable_variables()

In [ ]:
tf.trainable_variables()

### Напишем функцию, позволяющую генерить имена по затравке

*Что делается*

* Берется затравка (seed_phrase)
* Предсказывается вероятность появления следующего токена
* Следующий токен сэмплируется из распределения, предсказанного моделью
* Полученный токен добавляется к затравке
* Повторяем с шага 2

In [ ]:
def generate_sample(sess, seed_phrase=None, N=MAX_LEN, n_snippets=1):
    
    if seed_phrase is None:
        seed_phrase = ' '
    elif seed_phrase[0].isalpha():
        seed_phrase = ' ' + seed_phrase
    seed_phrase = seed_phrase.lower()
    seed_phrase = np.array([token_to_id[tok] for tok in seed_phrase])
    L = len(seed_phrase)
    snippets = []
    for _ in range(n_snippets):
        x = np.zeros(N)
        x[:len(seed_phrase)] = seed_phrase
        for n in range(N - L):
            feed_dict = {X: x[:L + n].reshape([1, -1]), lengths: [len(x)]}
            p = sess.run(last_word_probas, feed_dict=feed_dict).reshape(-1)
            ix = np.random.choice(np.arange(len(tokens)), p=p)
            x[L + n] = ix
        snippet = ''.join([id_to_token[idx] for idx in x])
        if ')' in snippet:
            upto = snippet.index(')')
            snippet = snippet[:upto]
        snippets.append(snippet.strip().capitalize())
    return snippets

In [ ]:
def print_pred(y_pred, k = 3):
    """
    k: сколько вывести 
    предсказаний модели среди всех y_pred
    
    """
    for i in range(k):
        print("".join( [id_to_token[t] for t in y_pred[i,:]]))

In [ ]:
s = tf.Session()
    
s.run(tf.global_variables_initializer())

n_epochs = 5
batches_per_epoch = 500
batch_size = 10
lr = 1e-2
for epoch in range(n_epochs):

    print(">>Generated: ", generate_sample(s, n_snippets=6))
    print("-------\n")
    avg_cost = 0
    for batch in range(batches_per_epoch):
        x_, y_, len_ = sample_batch(names_ix, batch_size)

        _, iloss, y_pred = s.run([train_op, loss, prediction], {X: x_,
                                                                y: y_,
                                                                lengths: len_,
                                                                learning_rate_ph: lr})
        avg_cost += iloss

    print("EPOCH: ", epoch)
    print("AVERAGE LOSS: ", avg_cost/batches_per_epoch)
    print(">>Predicted: ")
    print_pred(y_pred)

print(">>Generated: ", generate_sample(s, n_snippets=6))

In [ ]:
generate_sample(s, seed_phrase='Ann', n_snippets=10)

In [ ]:
generate_sample(s, seed_phrase='Sveta', n_snippets=6)

In [ ]:
generate_sample(s, seed_phrase='Eug', n_snippets=6)

In [ ]:
generate_sample(s, seed_phrase='Lu', n_snippets=19)

### Попробуйте использовать несколько слоев рекуррентных сетей

* Попробуйте поменять код модели, встроив туда модуль как в примере в следующей ячейке;
* Попробуйте использовать другие cells: LSTM, GRU;
* Попробуй генерировать твиты, скачав [датасет](http://study.mokoron.com) или какой угодно другой датасет


In [ ]:
# Как ненапряжно замутить глубокую рекуррентную нейросеть

n_neurons = 100
n_layers = 3

layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons,
                                      activation=tf.nn.relu) for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

In [ ]:
data_neg = pd.read_csv('negative.csv', encoding = 'utf-8', decimal = '"', error_bad_lines = False, delimiter=';', header = None)

In [ ]:
data_pos = pd.read_csv('positive.csv', encoding = 'utf-8', decimal = '"', error_bad_lines = False, delimiter=';', header = None)

In [ ]:
data_neg.head()

In [ ]:
data = pd.DataFrame()
data['seq'] = data_neg[3]

In [ ]:
#подключим необходимые библиотеки
#
import re
from nltk.corpus import stopwords
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

stops = set(stopwords.words("english")) | set(stopwords.words("russian"))
def review_to_wordlist(review):
    #1)
    review_text = re.sub("[^а-яА-Я]"," ", review)
    #2)
    words = review_text.lower().split()
    #3)
    words = [w for w in words if w in cv.vocabulary_]
    #3)
    #words = [w for w in words if not w in stops]
    #4)
    #words = [morph.parse(w)[0].normal_form for w in words ]
    return(words)

In [ ]:
data['seq_clear'] = data['seq'].apply(review_to_wordlist)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=100)
cv.fit(data['seq_clear'])
print(len(cv.vocabulary_))

In [ ]:
data['seq_clear'] = data['seq'].apply(review_to_wordlist)

In [ ]:
# Найдем все уникальные символы
# Без учета регистра

words_set = set()
for seq in data['seq_clear']:
    for word in seq:
        words_set.add(word)

token_set.add(' ')
words_set.add(')')
words = list(words_set)
words.sort()

print('n_tokens = ', len(words))

In [ ]:
print('n samples = ', len(data['seq_clear']))
for x in data['seq_clear'][::10000]:
    print(x)

In [ ]:
#!token_to_id = < словарь символов -> их id (index в tokens list)>
word_to_id = {t: i for i, t in enumerate(words)}

#!id_to_token = < словарь айдишников -> соответствующие символы >
id_to_word = {i: t for i, t in enumerate(words)}

In [ ]:
# Посмотрим какая максимальная длина у имени в этом датасете
MAX_LEN = min([60, max(list(map(len, data['seq_clear'])))])-1

print(MAX_LEN)

In [ ]:
words = list(data['seq_clear'])

In [ ]:
words_ix = list(map(lambda word: list(map(word_to_id.get, word + ')')), data['seq_clear']))


# Добьем нулями короткие имена до MAX_LEN и усечем слишком длинные
for i in range(len(words_ix)):
    words_ix[i] = words_ix[i][:MAX_LEN+1] #crop too long
    
    if len(words_ix[i]) < MAX_LEN+1:
        words_ix[i] += [word_to_id[" "]]*(MAX_LEN+1 - len(words_ix[i])) #pad too short
        
assert len(set(map(len, words_ix))) == 1

words_ix = np.array(words_ix)

In [ ]:
names = list(data['seq_clear'])

In [ ]:
start_token = [" "]
names = [start_token + name for name in names]

In [ ]:
# Найдем все уникальные символы
# Без учета регистра

token_set = set()
for name in names:
    for letter in name:
        token_set.add(letter)


token_set.add(')')
tokens = list(token_set)
tokens.sort()

print('n_tokens = ', len(tokens))

In [ ]:
#!token_to_id = < словарь символов -> их id (index в tokens list)>
token_to_id = {t: i for i, t in enumerate(tokens)}

#!id_to_token = < словарь айдишников -> соответствующие символы >
id_to_token = {i: t for i, t in enumerate(tokens)}

In [ ]:
token_to_id

In [ ]:
# Посмотрим какая максимальная длина у имени в этом датасете
MAX_LEN = min([60, max(list(map(len, names)))])-1

print(MAX_LEN)

In [ ]:
len_ = [len(_) for _ in s]

In [ ]:
s[23870]

In [ ]:
np.argmax(len_)

In [ ]:
names_ix = list(map(lambda name: list(map(token_to_id.get, name + [')'])), names))


# Добьем нулями короткие имена до MAX_LEN и усечем слишком длинные
for i in range(len(names_ix)):
    names_ix[i] = names_ix[i][:MAX_LEN+1] #crop too long
    
    if len(names_ix[i]) < MAX_LEN+1:
        names_ix[i] += [token_to_id[" "]]*(MAX_LEN+1 - len(names_ix[i])) #pad too short
        
assert len(set(map(len, names_ix))) == 1

names_ix = np.array(names_ix)

In [ ]:
names_ix[10]

In [ ]:
def sample_batch(data, batch_size):
    
    rows = data[np.random.randint(0, len(data), size=batch_size)]
    x = rows[:, :-1]
    y = rows[:, 1:]
    
    count = lambda r: np.sum([id_to_token[t] != ' ' for t in r])
    lengths = list(map(count, x))
    
    return x, y, lengths

In [ ]:
reset_graph()

# подразумевается, что размерность X [batch_size, max_length];
X = tf.placeholder(tf.int32, [None, None], name= 'X')
y = tf.placeholder(tf.int32, [None, None], name = 'y')
lengths = tf.placeholder(tf.int32, [None], name = 'lengths')
learning_rate_ph = tf.placeholder(dtype=tf.float32, shape=[])

n_neurons = 60
embedding_size = 8
vocabulary_size = len(tokens)

n_steps = MAX_LEN # Этот параметр совпадает
                  # с максимальной длиной последовательности, 
                  # которая может быть подана на вход
                  # иначе говоря, это unrollings

# для входной последовательности создаем матрицу эмбеддингов
embedding_mtx = tf.get_variable(name='embeddings', shape=[vocabulary_size, embedding_size])

# достаем из матрицы эмбеддингов нужные нам векторы X
embed = tf.nn.embedding_lookup(embedding_mtx, X)


cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.tanh)
rnn_outputs, states = tf.nn.dynamic_rnn(cell=cell, inputs=embed,
                                        sequence_length=lengths, dtype=tf.float32)

# получаем ненормированное распределение по классам 
# для каждого анроллинга в каждом сэмле в батче
pred_logits = tf.layers.dense(inputs=rnn_outputs, units=vocabulary_size, name='output_projection')

# кодируем one-hot классы, т.к. это тоже нужно функции лосса
labels_one_hot = tf.one_hot(y, depth=vocabulary_size, dtype=tf.float32)

# считаем илололосс
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=labels_one_hot,
    logits=pred_logits)
    
loss = tf.reduce_mean(stepwise_cross_entropy)

pred_probas = tf.nn.softmax(pred_logits)

# берем максимум по оси, соответствующей количеству классов
# получаем матрицу размера [batch_size, num_steps]
prediction = tf.argmax(pred_probas, axis=2)

train_op = tf.train.AdamOptimizer(learning_rate_ph).minimize(loss)

# берем распределение вероятностей только для последнего символа в каждом сэмпле
# это потребуется для генерации
last_word_probas = pred_probas[:, -1]

In [ ]:
def generate_sample(sess, seed_phrase=None, N=MAX_LEN, n_snippets=1):
    
    if seed_phrase is None:
        seed_phrase = [' ']
    elif seed_phrase[0].isalpha():
        seed_phrase = [' '] + seed_phrase
    #seed_phrase = [seed.lower() for seed in seed_phrase]
    seed_phrase = np.array([token_to_id[tok] for tok in seed_phrase])
    L = len(seed_phrase)
    snippets = []
    for _ in range(n_snippets):
        x = np.zeros(N)
        x[:len(seed_phrase)] = seed_phrase
        for n in range(N - L):
            feed_dict = {X: x[:L + n].reshape([1, -1]), lengths: [len(x)]}
            p = sess.run(last_word_probas, feed_dict=feed_dict).reshape(-1)
            ix = np.random.choice(np.arange(len(tokens)), p=p)
            x[L + n] = ix
        snippet = ' '.join([id_to_token[idx] for idx in x])
        if ')' in snippet:
            upto = snippet.index(')')
            snippet = snippet[:upto]
        snippets.append(snippet.strip().capitalize())
    return snippets

In [ ]:
def print_pred(y_pred, k = 3):
    """
    k: сколько вывести 
    предсказаний модели среди всех y_pred
    
    """
    for i in range(k):
        print(" ".join( [id_to_token[t] for t in y_pred[i,:]]))

In [ ]:
s = tf.Session()
    
s.run(tf.global_variables_initializer())

n_epochs = 5
batches_per_epoch = 500
batch_size = 30
lr = 1e-2
for epoch in range(n_epochs):

    print(">>Generated: ", generate_sample(s, n_snippets=6))
    print("-------\n")
    avg_cost = 0
    for batch in range(batches_per_epoch):
        x_, y_, len_ = sample_batch(names_ix, batch_size)

        _, iloss, y_pred = s.run([train_op, loss, prediction], {X: x_,
                                                                y: y_,
                                                                lengths: len_,
                                                                learning_rate_ph: lr})
        avg_cost += iloss

    print("EPOCH: ", epoch)
    print("AVERAGE LOSS: ", avg_cost/batches_per_epoch)
    print(">>Predicted: ")
    print_pred(y_pred)

print(">>Generated: ", generate_sample(s, n_snippets=6))

In [ ]:
s = tf.Session()
    
s.run(tf.global_variables_initializer())

n_epochs = 5
batches_per_epoch = 500
batch_size = 30
lr = 1e-2
for epoch in range(n_epochs):

    print(">>Generated: ", generate_sample(s, n_snippets=6))
    print("-------\n")
    avg_cost = 0
    for batch in range(batches_per_epoch):
        x_, y_, len_ = sample_batch(names_ix, batch_size)

        _, iloss, y_pred = s.run([train_op, loss, prediction], {X: x_,
                                                                y: y_,
                                                                lengths: len_,
                                                                learning_rate_ph: lr})
        avg_cost += iloss

    print("EPOCH: ", epoch)
    print("AVERAGE LOSS: ", avg_cost/batches_per_epoch)
    print(">>Predicted: ")
    print_pred(y_pred)

print(">>Generated: ", generate_sample(s, n_snippets=6))

In [ ]:
s = tf.Session()
    
s.run(tf.global_variables_initializer())

n_epochs = 25
batches_per_epoch = 500
batch_size = 30
lr = 1e-2
for epoch in range(n_epochs):

    print(">>Generated: ", generate_sample(s, n_snippets=6))
    print("-------\n")
    avg_cost = 0
    for batch in range(batches_per_epoch):
        x_, y_, len_ = sample_batch(names_ix, batch_size)

        _, iloss, y_pred = s.run([train_op, loss, prediction], {X: x_,
                                                                y: y_,
                                                                lengths: len_,
                                                                learning_rate_ph: lr})
        avg_cost += iloss

    print("EPOCH: ", epoch)
    print("AVERAGE LOSS: ", avg_cost/batches_per_epoch)
    print(">>Predicted: ")
    print_pred(y_pred)

print(">>Generated: ", generate_sample(s, n_snippets=6))

In [ ]:
generate_sample(s,seed_phrase =['люблю', 'света'] , n_snippets=6)

In [ ]:
s = tf.Session()
    
s.run(tf.global_variables_initializer())

n_epochs = 25
batches_per_epoch = 500
batch_size = 30
lr = 1e-2
for epoch in range(n_epochs):

    print(">>Generated: ", generate_sample(s, n_snippets=6))
    print("-------\n")
    avg_cost = 0
    for batch in range(batches_per_epoch):
        x_, y_, len_ = sample_batch(names_ix, batch_size)

        _, iloss, y_pred = s.run([train_op, loss, prediction], {X: x_,
                                                                y: y_,
                                                                lengths: len_,
                                                                learning_rate_ph: lr})
        avg_cost += iloss

    print("EPOCH: ", epoch)
    print("AVERAGE LOSS: ", avg_cost/batches_per_epoch)
    print(">>Predicted: ")
    print_pred(y_pred)

print(">>Generated: ", generate_sample(s, n_snippets=6))

In [ ]:
generate_sample(s, seed_phrase = ['хуй'] , n_snippets = 6)

In [2]:
with open('pushkin.txt', encoding = 'windows-1251') as f:
    s = f.read()

In [3]:
s = s.split('\n')

In [4]:
s

['Спасибо, что скачали книгу в бесплатной электронной библиотеке Royallib.ru: http://royallib.ru',
 '',
 'Все книги автора: http://royallib.ru/author/pushkin_aleksandr.html',
 '',
 'Эта же книга в других форматах: http://royallib.ru/book/pushkin_aleksandr/polnoe_sobranie_stihotvoreniy.html',
 '',
 'Приятного чтения!',
 '',
 '',
 '',
 '',
 'Александр Сергеевич Пушкин',
 '',
 'Полное собрание стихотворений',
 '',
 '',
 '',
 'Стихотворения 1809–1811\xa0гг',
 '',
 '',
 '* * *',
 '',
 "\t\tDis moi, pourquoi l'Escamoteur",
 '\t\tfist-il siffl&#233; par le parterre?',
 "\t\tH&#233;las! c'est que le pauvre auteur",
 "\t\tL'escamota de Moli&#233;re.",
 '',
 '',
 'Перевод',
 '',
 '\t\tСкажите мне, почему «Похититель»',
 '\t\tОсвистан партером?',
 '\t\tУвы, потому что бедный автор',
 '\t\tПохитил его у Мольера.',
 '',
 '',
 '* * *',
 '',
 '\t\tJe chante ce combat, que Toly remporta,',
 '\t\tO&#250; inaint guerrier p&#233;rit, o&#249; Paul se signala,',
 '\t\tNicolas Maturin et la belle Nitouche,'

In [ ]:
s = list(map(lambda x: re.sub("[^а-яёЁА-Я]"," ", x), s))

In [ ]:
s = [_.split() for _ in s]

In [ ]:
names = [_ for _ in s if len(_) > 2][4:]